# What will my Maths/Further Maths grades be?

This notebook implements the Edexcel aggregation rules to do two things:
1. Tell you what your Maths/Further Maths grades will be.
2. [coming soon] Present feasible grade combinations obtained by (permissible) modification of the Edexcel algorithm (e.g. an A in both Maths and Further Maths, instead of A* in Maths and B in Further Maths), so that you know what you can feasibly request Edexcel to do. 

For the sake of parsimony, this notebook will *not* compute AS-level grades (it only handles the overall A-level).

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import widgets, interact, interactive, fixed, interact_manual, HBox, VBox
from IPython.display import HTML, Javascript, display, Math, Latex

In [2]:
# get score from master list
def getscore (str, list):
    return list.index(str)



# get highest two further pure modules
def fpure_best(fpure):
    
    if fpure.index(min(fpure)) == 2: return "FP1", "FP2"
    elif fpure.index(min(fpure)) == 1: return "FP1", "FP3"
    else : return "FP2", "FP3"
    
    
# assign further maths modules in optimal way from leftover choices
def fmaths_best(fmaths,scores,a,b):
    fmaths_posib = ["FP1","FP2","FP3","M1","M2","M3","S1","S2","S3","D1"]
    temp = [0,1,2,3,4,5]
    fmaths_posib_scores = [0,0,0,0,0,0]
    result = fmaths
# remove modules already assigned to maths and fmaths, leaving 6 to work with
    fmaths_posib.remove(fmaths[0]) 
    fmaths_posib.remove(fmaths[1])
    fmaths_posib.remove(a)
    fmaths_posib.remove(b)
# sort remaining and assign best 4.
    for i in range (0,6):
        fmaths_posib_scores[i] = scores[1][getscore(fmaths_posib[i],scores[0])]
    fmaths_posib_sorted = [x for _,x in sorted(zip(fmaths_posib_scores,temp))]
    
    for i in range(0,4):
        result[i+2] = fmaths_posib[fmaths_posib_sorted[5-i]]
# remove modules just assigned       
    for i in range (0,4):
        fmaths_posib.remove(result[i+2])
# check for A* optimisation with a single swap if the current grade is A
    if grade_fmaths(result,scores) == 2:
        result_temp = result.copy()
        for i in range (0,4):
            if "1" in result_temp[5-i]: 
                result_temp[5-i] = fmaths_posib[1]
                break
        if grade_fmaths(result_temp,scores) == 1: result = result_temp
# check for A* optimisation with a double swap if the current grade is A
    if grade_fmaths(result,scores) == 2:
        result_temp = result.copy()
        index = 1;
        for i in range (0,4):
            if "1" in result_temp[5-i] and index == 1: 
                result_temp[5-i] = fmaths_posib[index]
                index = 0
            if "1" in result_temp[5-i] and index == 0: 
                result_temp[5-i] = fmaths_posib[index]
                break
        if grade_fmaths(result_temp,scores) == 1: result = result_temp
        
    return result


    
    
    
# compute maths grade
def grade_maths(maths,scores):
    
    maths_scores = [0,0,0,0,0,0]
    
    for i in range(0,6):
        maths_scores[i] = scores[1][getscore(maths[i], scores[0])]
    
    total = sum(maths_scores)
    star = maths_scores[2] + maths_scores[3]
    
    result = 0
    if total < 240: result = 7
    elif total < 300: result = 6
    elif total < 360: result = 5
    elif total < 420: result = 4
    elif total < 480: result = 3
    elif total >= 480 and star < 180: result = 2
    elif total >= 480 and star >= 180: result = 1
        
    return result



# compute further maths grade
def grade_fmaths(fmaths,scores):
    
    fmaths_scores = [0,0,0,0,0,0]
    for i in range(0,6):
        fmaths_scores[i] = scores[1][getscore(fmaths[i], scores[0])]
    
    total = sum(fmaths_scores)
    
    star_scores = fmaths_scores
    for i in range (0,6):
        if "1" in fmaths[i]: star_scores[i]=0
    star = sorted(star_scores)[-1] + sorted(star_scores)[-2] + sorted(star_scores)[-3]
    
    result = 0
    if total < 240: result = 7
    elif total < 300: result = 6
    elif total < 360: result = 5
    elif total < 420: result = 4
    elif total < 480: result = 3
    elif total >= 480 and star < 270: result = 2
    elif total >= 480 and star >= 270: result = 1
        
    return result



# compute grade for output
def grade_output(int):
    
    result = ''
    if int == 7: result = 'U'
    elif int == 6: result = 'E'
    elif int == 5: result = 'D'
    elif int == 4: result = 'C'
    elif int == 3: result = 'B'
    elif int == 2: result = 'A'
    elif int == 1: result = 'A*'
        
    return result

In [11]:
def aggregate(C1=0, C2=0, C3=0, C4=0, M1=0, M2=0, M3=0, S1=0, S2=0, S3=0, D1=0, FP1=0, FP2=0, FP3=0):
    
    
#first, get a master source. never edit this.
    scores = [["C1","C2","C3","C4","FP1","FP2","FP3","M1","M2","M3","S1","S2","S3","D1",""], 
              [C1, C2, C3, C4, FP1, FP2, FP3, M1, M2, M3, S1, S2, S3, D1,0]]
    
    
    
# insert non-negotiable scores
    maths = ["C1","C2","C3","C4", "", ""]
    maths_scores = [0,0,0,0,0,0]
    
    fmaths1, fmaths2 = fpure_best([FP1,FP2,FP3])
    fmaths = [fmaths1,fmaths2,"", "","",""]
    fmaths_orig = [fmaths1,fmaths2,"", "","",""]
    fmaths_scores = [0,0,0,0,0,0]
    
    
# compute totals of the math pairs and arrange in ascending order.
    maths_posib = [["M1", "M2"],["S1", "S2"], ["M1", "S1"], ["M1", "D1"], ["S1", "D1"]]
    temp = [0,1,2,3,4]
    maths_posib_scores = [0,0,0,0,0]
    
    for i in range (0,5):
        maths_posib_scores[i] = scores[1][getscore(maths_posib[i][0],scores[0])] + scores[1][getscore(maths_posib[i][1],scores[0])]
    maths_posib_sorted = [x for _,x in sorted(zip(maths_posib_scores,temp))]
    
    
# assign best pair to maths then check if reoptimisation is possible.
    maths[4] = maths_posib[maths_posib_sorted[4]][0]
    maths[5] = maths_posib[maths_posib_sorted[4]][1]
    fmaths = fmaths_best(fmaths_orig,scores,maths[4],maths[5])
    
    mgrade = grade_maths(maths,scores)
    fgrade = grade_fmaths(fmaths,scores)
    
    if mgrade < fgrade:
        maths_temp = ["C1","C2","C3","C4", "", ""]
        fmaths_temp = fmaths_orig.copy()
        for i in range (0,4):
            maths_temp[4] = str(maths_posib[maths_posib_sorted[3-i]][0])
            maths_temp[5] = str(maths_posib[maths_posib_sorted[3-i]][1])
            fmaths_temp = fmaths_best(fmaths_temp,scores,maths_temp[4],maths_temp[5])
            
            mgrade_temp = grade_maths(maths_temp,scores)
            fgrade_temp = grade_fmaths(fmaths_temp,scores)
        
            if mgrade_temp > mgrade: 
                break
            elif mgrade_temp <= mgrade and fgrade_temp < fgrade:
                maths = maths_temp.copy()
                fmaths = fmaths_temp.copy()

# generate all feasible grades
    maths_posib2 = [["M1", "M2"],["S1", "S2"], ["M1", "S1"], ["M1", "D1"], ["S1", "D1"]]
    maths_feasible = ['','','','','']
    fmaths_feasible = ['','','','','']
    for i in range(0,5):
        maths_tograde = ["C1","C2","C3","C4", maths_posib2[i][0], maths_posib2[i][1]]
        fmaths_tograde = [fmaths1,fmaths2,"", "","",""]
        fmaths_tograde = fmaths_best(fmaths_tograde,scores,maths_tograde[4],maths_tograde[5])
        maths_feasible[i] = grade_output(grade_maths(maths_tograde, scores))
        fmaths_feasible[i] = grade_output(grade_fmaths(fmaths_tograde, scores))
        
    combos = ["M1+M2","S1+S2","M1+S1","M1+D1","S1+D1"]
    
    
#format and display output
    for i in range(0,6):
        maths_scores[i] = scores[1][getscore(maths[i], scores[0])]
        
    for i in range(0,6):
        fmaths_scores[i] = scores[1][getscore(fmaths[i], scores[0])]
    
    maths_output = pd.DataFrame({'Module': maths, 'Score':maths_scores})
    fmaths_output = pd.DataFrame({'Module': fmaths, 'Score':fmaths_scores})
    combos_output = pd.DataFrame({'Maths grade':maths_feasible,'Further grade':fmaths_feasible, 'To maths': combos}, columns=['To maths', 'Maths grade', 'Further grade'])
    
    
    print("Maths grade: " + grade_output(grade_maths(maths, scores)))
    print(maths_output.to_string(index=False))
    print("")
    print("Further maths grade: " + grade_output(grade_fmaths(fmaths, scores)))
    print(fmaths_output.to_string(index=False))
    print("")
    print("Feasible combos:")
    print(combos_output.to_string(index=False))

# Instructions

1. Use the sliders to input your marks.
2. Alternatively, tap on the number and key in your mark. If you do, don't forget to press 'Enter'.
3. Enjoy the results (or not).

This is not meant to work on mobile devices. Use a PC.

In [12]:
C1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C1:')
C2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C2:')
C3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C3:')
C4 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C4:')
C = HBox([C1,C2,C3,C4],layout={'width':'100%'})

FP1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP1:')
FP2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP2:')
FP3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP3:')
FP = HBox([FP1,FP2,FP3],layout={'width':'75%'})

M1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M1:')
M2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M2:')
M3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M3:')
M = HBox([M1,M2,M3], layout={'width':'75%'})

S1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S1:')
S2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S2:')
S3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S3:')
S = HBox([S1,S2,S3],layout={'width':'75%'})

D1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='D1:')
D = HBox([D1], layout={'width':'25%'})
                  
P1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='')
P = HBox([P1], layout={'width':'0%'})

your_marks = widgets.VBox([C,FP,M,S,D,P])
your_grades = widgets.interactive_output(aggregate, {'C1':C1,'C2':C2,'C3':C3,'C4':C4,'FP1':FP1,'FP2':FP2,'FP3':FP3,'M1':M1,'M2':M2,'M3':M3,'S1':S1,'S2':S2,'S3':S3,'D1':D1})

display(your_marks)
display(your_grades)

A Jupyter Widget

A Jupyter Widget

## Algorithm (based on the [Edexcel Aggregation Rules](http://furthermaths.org.uk/docs/Aggregation%20rules%20for%20HEIs%20~%20Version%203%20January%202013.pdf))

As a guiding philosophy, the aggregation rules aim to do the following:

1. Maximise the Maths grade, then the Further Maths grade.
2. Maximise the Maths UMS, then the Further Maths UMS.

Just to be clear, the order of priority above implies that lowering your Maths UMS **(without reducing the grade)** in order to increase your Further Maths grade *is* acceptable, since grades are maximised before UMS. However, if increasing your Further Maths grade comes at the cost of reducing your Maths grade, that would *not* be acceptable, since the Math grade is maximised before the Further Maths grade.

The **algorithm** is as follows:
1. Allocate C1, C2, C3, C4 to Maths (non-negotiable).
2. Allocate the highest 2 of (FP1, FP2, FP3) to Further Maths (non-negotiable).
3. Allocate the highest (total UMS) of the following 6 pairs to Maths:
    -  M1+M2
    -  S1+S2
    -  D1+D2
    -  M1+S1
    -  M1+D1
    -  S1+D1. 
4. Allocate the highest 4 of the remaining modules to Further Math.
5. Check if the Further Maths **grade can be improved** by removing one of the 4-highest from Step 4 and replacing it with an A2 module (e.g. replacing a 97 in D1 with a 90 in M2 may lower total UMS but increase the grade from A to A*). If it can, then the re-allocation is performed.
6. Check if the Further Maths grade can be improved on by allocating the best pair from step (3) to Further Maths and the second best pair to Maths, **subject to not lowering the Maths grade**. If can, then the re-allocation is performed. If it cannot, then the allocation is now optimal.